In [1]:
!pip install flair

     |████████████████████████████████| 184kB 4.8MB/s 
     |████████████████████████████████| 798kB 23.3MB/s 
     |████████████████████████████████| 184kB 46.9MB/s 
     |████████████████████████████████| 645kB 50.6MB/s 
     |████████████████████████████████| 1.0MB 39.5MB/s 
     |████████████████████████████████| 778kB 42.9MB/s 
     |████████████████████████████████| 860kB 46.2MB/s 
     |████████████████████████████████| 1.0MB 29.1MB/s 
     |████████████████████████████████| 348kB 42.2MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116679 sha256=4489a40f4c06133ade6a262ce1bc261a461c221daf5da88c9854edffe471f5fe
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Created wheel for segtok: filename=segtok-1.5.7-cp36-none-any.whl size=23258 sha256=90c9f0918fe5aa3c7d4175809616796d7e3de6b24bc3ae13c120582e181a2601
  Stored in directory: /root/.cache/pip/wheels/15/ee/a8/6112173f1386d33eebedb3f73429cfa41a4

In [0]:
import codecs

def read_ruscorpora(filename):
    sent = [[]]
    with codecs.open(filename, encoding='utf-8') as inp_file:
        for line in inp_file:
            line = line.strip('\r\n ')
            if '\t' in line:
                line = line[:line.find('\t')]
            if line.startswith('#'):
                continue
            if not line:
                sent.append([])
                continue
            wordform, lemma, gram = line.rsplit('/', 2)
            pos = gram[:gram.find('=')] if '=' in gram else gram
            pos = pos.split(',')[0]
            sent[-1].append((wordform, pos, lemma, gram))
    
    return sent

In [0]:
corpus = read_ruscorpora('ruscorpora.parsed.txt')

In [0]:
gran1 = int(len(corpus)*0.3)
gran2 = int(len(corpus)*0.4)
gran3 = int(len(corpus)*0.5)

with open('sample_data/train.txt', 'a') as f:
  for i in range(0, gran1):
    for sentence in corpus[i]:
      for word in sentence:
        for symb in word:
          f.write(symb)
        f.write(' ')
      f.write('\n')

In [0]:
with open('sample_data/test.txt', 'a') as f:
  for i in range(gran1, gran2):
    for sentence in corpus[i]:
      for word in sentence:
        for symb in word:
          f.write(symb)
        f.write(' ')
      f.write('\n')

In [0]:
with open('sample_data/dev.txt', 'a') as f:
  for i in range(gran2, int(len(corpus)*0.9)):
    for sentence in corpus[i]:
      for word in sentence:
        for symb in word:
          f.write(symb)
        f.write(' ')
      f.write('\n')

In [16]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# this is the folder in which train, test and dev files reside
data_folder = 'sample_data'

# define columns
columns = {0: 'text', 1: 'pos', 2: 'token', 3: 'morph'}

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='dev.txt')

2019-10-20 18:54:12,180 Reading data from sample_data
2019-10-20 18:54:12,184 Train: sample_data/train.txt
2019-10-20 18:54:12,187 Dev: sample_data/dev.txt
2019-10-20 18:54:12,188 Test: sample_data/test.txt


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [18]:
import flair.datasets
from flair.data import Corpus, Sentence
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, FlairEmbeddings
from flair.trainers import ModelTrainer
from flair.models import SequenceTagger
from typing import List

tag_type = 'morph'

# словарь тегов по корпусу
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)

# выбираем эмбеддинги
embedding_types: List[TokenEmbeddings] = [
    FlairEmbeddings('multi-forward')
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# описываем теггер
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

# описываем обучение
trainer: ModelTrainer = ModelTrainer(tagger, corpus)
trainer.train('drive/My Drive/govno',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=3,
              checkpoint=True)

[b'<unk>', b'O', b'A=n,sg,nom,plen', b'S,n,inan=sg,nom', b'A=f,sg,gen,plen', b'S,f,inan=sg,gen', b'V,ipf,tran=sg,act,praes,3p,indic', b'V,pf,tran=inf,act', b'S,m,inan=pl,acc', b'A=m,sg,gen,plen', b'S,m,inan=sg,gen', b'PR', b'V,ipf,intr,act=partcp,pl,gen,praet,plen', b'S,m,inan,0=sg,loc', b'S,f,inan=sg,loc', b'S,f,inan,0=sg,gen', b'S,m,anim=pl,nom', b'S,m,anim=pl,gen', b'V,ipf,intr,act=pl,fut,3p,indic', b'V,ipf,intr,act=inf', b'S,n,inan=sg,ins', b'A=n,sg,gen,plen', b'S,n,inan=sg,gen', b'CONJ', b'A=f,sg,ins,plen', b'S,f,inan=sg,ins', b'A=pl,loc,plen', b'ADV-PRO', b'S,n,inan=pl,loc', b'S,n,inan=sg,loc', b'V,ipf,tran=partcp,m,sg,gen,act,praes,plen', b'V,pf,tran=m,sg,act,praet,indic', b'ADV', b'S,m,anim=sg,nom', b'S,n,inan=sg,dat', b'S,persn,m,anim=sg,nom', b'S,famn,m,anim=sg,nom', b'A-PRO', b'S,n,inan=pl,dat', b'A,brev=pl', b'A-PRO=n,sg,acc,inan', b'A=n,sg,acc,inan,plen', b'S-PRO,n,sg=nom', b'V,ipf,intr,act=n,sg,praet,indic', b'S,m,inan=pl,gen', b'V,ipf,intr,med=sg,praes,3p,indic', b'A=pl,

100%|██████████| 73034300/73034300 [00:05<00:00, 13768518.45B/s]

2019-10-20 18:57:30,093 copying /tmp/tmpp955e8zt to cache at /root/.flair/embeddings/lm-multi-forward-v0.1.pt


2019-10-20 18:57:30,169 removing temp file /tmp/tmpp955e8zt
2019-10-20 18:57:33,539 ----------------------------------------------------------------------------------------------------
2019-10-20 18:57:33,543 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.5, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=2048, out_features=2048, bias=True)
  (rnn): LSTM(2048, 256, bidirectional=True)
  (linear): Linear(in_features=512, out_features=2583, bias=True)
)"
2019-10-20 18:57:33,546 ----------------------------------------------------------------------------------------------------
2019-10-20 18:57:33,551 Corpus: "Corpus: 1 train + 1 dev + 1 test 

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flair/trainers/trainer.py", line 273, in train
    loss = self.model.forward_loss(batch)
  File "/usr/local/lib/python3.6/dist-packages/flair/models/sequence_tagger_model.py", line 344, in forward_loss
    features = self.forward(data_points)
  File "/usr/local/lib/python3.6/dist-packages/flair/models/sequence_tagger_model.py", line 405, in forward
    self.embeddings.embed(sentences)
  File "/usr/local/lib/python3.6/dist-packages/flair/embeddings.py", line 149, in embed
    embedding.embed(sentences)
  File "/usr/local/lib/python3.6/dist-packages/flair/embeddings.py", line 81, in embed
    self._add_embeddings_internal(sentences)
  File "/usr/local/lib/python3.6/dist-packages/flair/embeddings.py", line 1822, in _add_embeddings_internal
    sentences_padded, self.chars_per_chunk
  File "/usr/local/lib/python3.6/dist-packages/flair/models/language_model.py", line 131, in get_represent

FileNotFoundError: ignored

In [0]:
sentence = Sentence('Я люблю спать.')

In [44]:
tagger.predict(sentence)
print(sentence)
print(sentence.to_tagged_string())

Sentence: "Я люблю спать." - 3 Tokens
Я <S-PRO,m,anim,sg=nom> люблю <S-PRO,m,sg,3p=loc> спать. <V,intr,ipf,med=imper,sg,2p>
